# DATA LOADING AND PREPROCESSING

In [1]:
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, TFBertModel
import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv('/kaggle/input/sarcasm/train-balanced-sarcasm.csv')
df.head()

,label,comment,author,subreddit,score,ups,downs,date,created_utc,parent_comment
0,0,NC and NH.,Trumpbart,politics,2,-1,-1,2016-10,2016-10-16 23:55:23,"Yeah, I get that argument. At this point, I'd ..."
1,0,You do know west teams play against west teams...,Shbshb906,nba,-4,-1,-1,2016-11,2016-11-01 00:24:10,The blazers and Mavericks (The wests 5 and 6 s...
2,0,"They were underdogs earlier today, but since G...",Creepeth,nfl,3,3,0,2016-09,2016-09-22 21:45:37,They're favored to win.
3,0,"This meme isn't funny none of the ""new york ni...",icebrotha,BlackPeopleTwitter,-8,-1,-1,2016-10,2016-10-18 21:03:47,deadass don't kill my buzz
4,0,I could use one of those tools.,cush2push,MaddenUltimateTeam,6,-1,-1,2016-12,2016-12-30 17:00:13,Yep can confirm I saw the tool they use for th...


In [3]:
df.shape

(1010826, 10)

In [4]:
df = df[:5000]
df = df[['comment', 'label']]
df.head()

,comment,label
0,NC and NH.,0
1,You do know west teams play against west teams...,0
2,"They were underdogs earlier today, but since G...",0
3,"This meme isn't funny none of the ""new york ni...",0
4,I could use one of those tools.,0


In [5]:
df.shape

(5000, 2)

In [6]:
df.isnull().sum()

comment    1
label      0
dtype: int64

In [7]:
df.dropna(inplace = True)
df.isna().sum()

comment    0
label      0
dtype: int64

In [8]:
df['comment'] = df['comment'].str.replace(r'[^a-zA-Z\s]','', regex = True)

In [9]:
def lowercase(text):
    return text.lower()

df['comment'] = df['comment'].apply(lowercase)

In [10]:
df.head()

,comment,label
0,nc and nh,0
1,you do know west teams play against west teams...,0
2,they were underdogs earlier today but since gr...,0
3,this meme isnt funny none of the new york nigg...,0
4,i could use one of those tools,0


# TOKENIZATION

In [11]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [12]:
def tokenize_data(text, max_length = 100):
    return tokenizer(
        text.tolist(),
        max_length = max_length,
        truncation = True,
        padding = 'max_length',
        return_tensors = 'np'
    )

tokenized_data = tokenize_data(df['comment'])

In [13]:
tokenized_data

{'input_ids': array([[  101, 13316,  1998, ...,     0,     0,     0],
       [  101,  2017,  2079, ...,     0,     0,     0],
       [  101,  2027,  2020, ...,     0,     0,     0],
       ...,
       [  101,  3492,  2204, ...,     0,     0,     0],
       [  101,  2129,  2079, ...,     0,     0,     0],
       [  101,  2059,  3013, ...,     0,     0,     0]]), 'token_type_ids': array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]]), 'attention_mask': array([[1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       ...,
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0]])}

# TRAIN-TEST SPLIT

In [14]:
x = tokenized_data['input_ids']
y = df['label']

In [15]:
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.2,random_state=42)

In [16]:
x_train.shape, x_test.shape

((3999, 100), (1000, 100))

THIS 100 VALUE IS COMING DUE TO PADDING, WHICH MEANS THE MAX_LENGTH THAT WE HAVE FIXED

# BUILDING THE MODEL
THIS MODEL IS BUILT BY TAKING THE REFERENCE OF A RESEARCH PAPER WHICH IS INCLUDED IN THE INPUT

In [17]:
#LOADING THE PRE-TRAINED BERT MODEL
bert_model = TFBertModel.from_pretrained('bert-base-uncased')

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

In [18]:
class HierarchicalBERT(tf.keras.Model):
    def __init__(self, bert_model, lstm_units, cnn_filters, dense_units):
        super(HierarchicalBERT, self).__init__()
        self.bert = bert_model
        
        #SENTENCE ENCODING LAYER
        self.dense_sentence = tf.keras.layers.Dense(768, activation = 'relu')
        
        #CONTEXT SUMMARIZATION LAYER
        self.mean_pooling = tf.keras.layers.GlobalAveragePooling1D()
        
        #CONTEXT ENCODER LAYER
        self.bilstm_encoder = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(lstm_units, return_sequences = True))
        
        #CNN LAYER
        self.conv = tf.keras.layers.Conv1D(cnn_filters, 2, activation = 'relu')
        self.pool = tf.keras.layers.GlobalMaxPooling1D()
        
        #FULLY CONNECTED LAYER
        self.dense_fc = tf.keras.layers.Dense(dense_units, activation = 'relu')
        self.output_layer = tf.keras.layers.Dense(1, activation = 'sigmoid')

    def call(self, inputs):
        #BERT EMBEDDINGS
        bert_output = self.bert(inputs)[0]
        
        #SENTENCE ENCODING LAYER
        sentence_encoded = self.dense_sentence(bert_output)
        
        #CONTEXT SUMMARIZATION LAYER
        context_summarized = self.mean_pooling(sentence_encoded)
        
        #EXPAND DIMENSION TO MATCH THE INPUT SHAPE REQUIRED BY LSTM
        context_summarized = tf.expand_dims(context_summarized,1)
        
        #CONTEXT ENCODER LAYER
        context_encoded = self.bilstm_encoder(context_summarized)
        
        #SQUEEZE THE INPUT SHAPE TO MATCH THE SECOND DIMENSIONS REQUIRED BY CONV1D
        context_encoded_squeezed = tf.squeeze(context_encoded, axis = 1)
        
        #ADD THE CHANNEL DIMENSION TO MATCH THE INPUT SHAPE REQUIRED BY CONV1D
        context_encoded_expanded = tf.expand_dims(context_encoded_squeezed, axis = -1)
        
        #CNN LAYER
        conv_output = self.conv(context_encoded_expanded)
        pooled_output = self.pool(conv_output)
        
        #FULLY CONNECTED LAYER
        dense_output = self.dense_fc(pooled_output)
        
        #OUTPUT LAYER
        final_output = self.output_layer(dense_output)
        return final_output
     
        


In [19]:
#DEFINING THE HIERARICAL BERT MODEL
model = HierarchicalBERT(bert_model, lstm_units = 128, cnn_filters = 64, dense_units = 32)

In [20]:
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [21]:
model.fit(x_train, y_train, epochs = 3,batch_size = 32)

Epoch 1/3
125/125 [==============================] - 2138s 16s/step - loss: 0.4951 - accuracy: 0.8082
Epoch 2/3
125/125 [==============================] - 2006s 16s/step - loss: 0.4852 - accuracy: 0.8112
Epoch 3/3
125/125 [==============================] - 1967s 16s/step - loss: 0.4856 - accuracy: 0.8112


In [22]:
loss, accuracy = model.evaluate(x_test, y_test)
print(f'Model Accuracy: {accuracy*100}')

32/32 [==============================] - 161s 5s/step - loss: 0.4836 - accuracy: 0.8120
Model Accuracy: 81.19999766349792
